In [1]:
import sys
sys.path.append('.')

In [2]:
import yaml
import numpy as np
import pandas as pd

In [3]:
from competition import Repository

In [4]:
with open('configs/lag_pipeline.yaml', 'r') as f:
    config = yaml.load(f, yaml.Loader)
config

{'name': 'lag_pipeline',
 'quality': -1,
 'repository': {'impute_transformer': 'sklearn.impute.SimpleImputer',
  'ord_transformer': 'sklearn.preprocessing.OrdinalEncoder',
  'var_selector': 'sklearn.feature_selection.VarianceThreshold',
  'cat_boost_classifier': 'catboost.CatBoostClassifier',
  'timeseries_spliter': 'sklearn.model_selection.TimeSeriesSplit'},
 'warehouse': {'data_path': 'input/',
  'create_args': {'use_expanded_train': True,
   'use_geo': True,
   'use_era5': True,
   'era5_metrics': '*',
   'era5_years': '*'}},
 'featurise': {'calcers': {'dates_features': {},
   'geo_cat_features': {},
   'geo_neighbors_features': {'count_neighbors': 5},
   'grib_features': {'metrics': '*',
    'pooling_size': 3,
    'lags': [0, 1, 2, 3],
    'agg_funcs': ['max', 'mean']},
   'lags_features': {'lags': [1, 2, 3, 4, 5, 6, 7]},
   'target_base': {}}},
 'pipeline': {'name': 'lags_pipeline',
  'seed': 110894,
  'split_dt': '2021-01-01',
  'estimators': {'transformers': [{'name': 'geo_place

In [5]:
repo = Repository()
for name, obj_name in config['repository'].items():
    repo.register(name, obj_name)
repo._objects

{'dates_features': competition.featurise.calcers.DatesFeaturesCalcer,
 'geo_cat_features': competition.featurise.calcers.GeoCatFeaturesCalcer,
 'geo_neighbors_features': competition.featurise.calcers.GeoNeighborsFeaturesCalcer,
 'grib_features': competition.featurise.calcers.GribFeaturesCalcer,
 'lags_features': competition.featurise.calcers.LagsFeaturesCalcer,
 'target_base': competition.featurise.calcers.TargetBaseCalcer,
 'dummy_selector': competition.estimators.selectors.DummySelector,
 'baseline_pipeline': competition.pipeline.pipelines.BaselinePipeline,
 'lags_pipeline': competition.pipeline.pipelines.LagsPipeline,
 'impute_transformer': sklearn.impute._base.SimpleImputer,
 'ord_transformer': sklearn.preprocessing._encoders.OrdinalEncoder,
 'var_selector': sklearn.feature_selection._variance_threshold.VarianceThreshold,
 'cat_boost_classifier': catboost.core.CatBoostClassifier,
 'timeseries_spliter': sklearn.model_selection._split.TimeSeriesSplit}

# Lags Pipeline

In [6]:
features_df = pd.read_csv('features/lag_pipeline_features.csv')
features_df.info()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053086 entries, 0 to 1053085
Columns: 226 entries, dt to infire_lag_day_7
dtypes: float64(212), int64(12), object(2)
memory usage: 1.8+ GB


In [7]:
pipeline = repo.get_object(config['pipeline']['name'])(config['pipeline']['seed'])
pipeline

In [8]:
pipeline.fit(features_df, config['pipeline'], repo)

In [20]:
test_df = features_df.loc[:, ['dt', 'grid_index'] + pipeline.features].sort_values(['grid_index', 'dt']).iloc[:100, :]
test_df.head()

,dt,grid_index,day_of_week,evaporation1_e_ws3_max_lag_0,evaporation1_e_ws3_max_lag_1,evaporation1_e_ws3_max_lag_2,evaporation1_e_ws3_max_lag_3,evaporation1_e_ws3_mean_lag_0,evaporation1_e_ws3_mean_lag_1,evaporation1_e_ws3_mean_lag_2,...,wind_u10_ws3_mean_lag_2,wind_u10_ws3_mean_lag_3,wind_v10_ws3_max_lag_0,wind_v10_ws3_max_lag_1,wind_v10_ws3_max_lag_2,wind_v10_ws3_max_lag_3,wind_v10_ws3_mean_lag_0,wind_v10_ws3_mean_lag_1,wind_v10_ws3_mean_lag_2,wind_v10_ws3_mean_lag_3
0,2018-05-01,143,1,-0.000402,-0.000350,-0.000106,-0.000079,-0.002585,-0.002070,-0.001379,...,-0.695887,-0.496770,1.392036,1.363419,0.802750,0.300106,1.140572,0.990657,0.175227,-0.127954
1,2019-03-01,143,4,-0.000195,-0.000091,-0.000054,-0.000123,-0.000677,-0.000239,-0.000245,...,0.518565,1.286392,0.217300,0.654205,0.674881,0.662689,-0.816798,0.525136,0.327713,-0.432688
2,2019-05-10,143,4,-0.002009,-0.002875,-0.002215,-0.001940,-0.002299,-0.003592,-0.002940,...,0.998850,0.592316,1.322006,1.089293,1.162502,0.729484,1.059107,0.908507,0.744371,0.595511
3,2019-11-11,143,0,-0.000355,-0.000372,-0.000330,-0.000264,-0.000693,-0.000716,-0.000741,...,-0.606679,-0.465627,-0.200333,0.344360,-0.203720,-0.253479,-0.646378,-0.327596,-0.642644,-0.617249
4,2020-05-29,143,4,-0.003706,-0.003764,-0.003584,-0.003586,-0.004024,-0.004003,-0.003824,...,0.204870,0.590007,1.294907,1.098717,1.300857,1.668045,0.954046,0.982424,1.230626,1.279943


In [18]:
any(test_df.groupby('grid_index')['dt'].size() > 1)

True

In [59]:
test_df[test_df.columns[test_df.columns.str.contains('lag_day')]].describe()

,infire_lag_day_1,infire_lag_day_2,infire_lag_day_3,infire_lag_day_4,infire_lag_day_5,infire_lag_day_6,infire_lag_day_7
count,100.00,100.0,100.0,100.0,100.0,100.0,100.0
mean,0.01,0.0,0.0,0.0,0.0,0.0,0.0
std,0.10,0.0,0.0,0.0,0.0,0.0,0.0
min,0.00,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.00,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.00,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.00,0.0,0.0,0.0,0.0,0.0,0.0
max,1.00,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
lag_cols[:-1]

['infire_lag_day_1',
 'infire_lag_day_2',
 'infire_lag_day_3',
 'infire_lag_day_4',
 'infire_lag_day_5',
 'infire_lag_day_6']

In [86]:
prediction_df = pd.DataFrame(test_df[['dt', 'grid_index']])
for i in range(1, 9):
    prediction_df[f'infire_day_{i}'] = np.nan

key_cols = ['dt', 'grid_index']
targets = [f'infire_day_{i}' for i in range(1, 9)]
lag_cols = [f'infire_lag_day_{i}' for i in range(1, 8)]

# temp_df = test_df.copy()
while True: 
    #get first
    iter_df = (test_df
               .merge(prediction_df.loc[prediction_df[targets].isna().all(axis=1), key_cols],
                      how='inner',
                      on=key_cols)  
               .sort_values(['grid_index', 'dt'])
               .drop_duplicates('grid_index', keep='first'))
    if len(iter_df) == 0:
        break
    
    #update lags
    for i in range(1, 8):
        prediction_df['dt'] = (pd.to_datetime(iter_df['dt']) - pd.DateOffset(i)).dt.strftime('%Y-%m-%d')
        iter_df = iter_df.merge(prediction_df[key_cols + ['infire_day_1']],
                                how='left',
                                on=key_cols)
        iter_df[f'infire_lag_day_{i}'] = iter_df['infire_day_1'].combine_first(iter_df[f'infire_lag_day_{i}'])
        del iter_df['infire_day_1']
    prediction_df.loc[:, 'dt'] = test_df['dt']

    #prediction 
    for i in range(1, 9):
        iter_df[f'infire_day_{i}'] = pipeline.model.predict(iter_df.loc[:, pipeline.features])
        iter_df.loc[:, lag_cols] = iter_df[[f'infire_day_{i}'] + lag_cols[:-1]].values

    prediction_df = prediction_df.merge(iter_df[key_cols + targets], how='left', on=key_cols)
    for target in targets:
        prediction_df.loc[:, target] = prediction_df[f'{target}_y'].combine_first(prediction_df[f'{target}_x'])
        del prediction_df[f'{target}_x']
        del prediction_df[f'{target}_y']

In [87]:
prediction_df.head()

,dt,grid_index,infire_day_1,infire_day_2,infire_day_3,infire_day_4,infire_day_5,infire_day_6,infire_day_7,infire_day_8
0,2018-05-01,143,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2019-03-01,143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-05-10,143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-11-11,143,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,2020-05-29,143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
prediction_df = pd.DataFrame(test_df[['dt', 'grid_index']])
for i in range(1, 9):
    prediction_df[f'infire_day_{i}'] = np.nan
prediction_df.head()

,dt,grid_index,infire_day_1,infire_day_2,infire_day_3,infire_day_4,infire_day_5,infire_day_6,infire_day_7,infire_day_8
0,2018-05-01,143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-03-01,143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-05-10,143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-11-11,143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-05-29,143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
key_cols = ['dt', 'grid_index']
targets = [f'infire_day_{i}' for i in range(1, 9)]
lag_cols = [f'infire_lag_day_{i}' for i in range(1, 8)]

In [100]:
iter_df = (test_df
           .merge(prediction_df.loc[prediction_df[targets].isna().all(axis=1), key_cols],
                  how='inner',
                  on=key_cols)  
           .sort_values(['grid_index', 'dt'])
           .drop_duplicates('grid_index', keep='first'))
iter_df[lag_cols].head()

,infire_lag_day_1,infire_lag_day_2,infire_lag_day_3,infire_lag_day_4,infire_lag_day_5,infire_lag_day_6,infire_lag_day_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
for i in range(1, 8):
    prediction_df['dt'] = (pd.to_datetime(iter_df['dt']) - pd.DateOffset(i)).dt.strftime('%Y-%m-%d')
    iter_df = iter_df.merge(prediction_df[key_cols + ['infire_day_1']],
                            how='left',
                            on=key_cols)
    iter_df[f'infire_lag_day_{i}'] = iter_df['infire_day_1'].combine_first(iter_df[f'infire_lag_day_{i}'])
    del iter_df['infire_day_1']
prediction_df.loc[:, 'dt'] = test_df['dt']
iter_df[lag_cols].head()

,infire_lag_day_1,infire_lag_day_2,infire_lag_day_3,infire_lag_day_4,infire_lag_day_5,infire_lag_day_6,infire_lag_day_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0


for i in range(1, 9):
    iter_df[f'infire_day_{i}'] = pipeline.model.predict(iter_df.loc[:, pipeline.features])
    iter_df.loc[:, lag_cols] = iter_df[[f'infire_day_{i}'] + lag_cols[:-1]].values
iter_df[lag_cols + targets].head()

In [ ]:
i = 1
iter_df[f'infire_day_{i}'] = pipeline.model.predict(iter_df.loc[:, pipeline.features])
iter_df[lag_cols + [f'infire_day_{j}' for j in range(1, i + 1)]].head()

In [ ]:
iter_df.loc[:, lag_cols] = iter_df[[f'infire_day_{i}'] + lag_cols[:-1]].values
iter_df[lag_cols + [f'infire_day_{j}' for j in range(1, i + 1)]].head()

# Lags features

In [7]:
from competition.warehouse import Warehouse
from competition.featurise import Featuriser

In [8]:
engine = Warehouse('input/').create(use_expanded_train=True, use_geo=False, use_era5=False)
features_df = Featuriser(engine, repo).get_features({'lags_features': {'lags': [1, 2, 3, 4, 5, 6, 7]}})
features_df.info()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1053086 entries, 0 to 1053085
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   dt                1053086 non-null  object 
 1   grid_index        1053086 non-null  int64  
 2   infire_lag_day_1  1053086 non-null  float64
 3   infire_lag_day_2  1053086 non-null  float64
 4   infire_lag_day_3  1053086 non-null  float64
 5   infire_lag_day_4  1053086 non-null  float64
 6   infire_lag_day_5  1053086 non-null  float64
 7   infire_lag_day_6  1053086 non-null  float64
 8   infire_lag_day_7  1053086 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 80.3+ MB


In [15]:
other_features_df = pd.read_csv('features/more_target_features.csv')
other_features_df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,dt,grid_index,month,week,day_of_week,geo_population,geo_place,geo_cn5_city_mean_distance,geo_cn5_city_max_distance,geo_cn5_city_min_distance,...,wind_tp_ws3_mean_lag_3,infire_day_1,infire_day_2,infire_day_3,infire_day_4,infire_day_5,infire_day_6,infire_day_7,infire_day_8,infire_day_num
0,2018-05-01,143,5,18,1,NaN,NaN,0.344685,0.378005,0.288359,...,0.002256,0,0,0,0,0,0,0,0,0.0
1,2019-03-01,143,3,9,4,NaN,NaN,0.344685,0.378005,0.288359,...,0.000011,0,0,0,0,0,0,0,0,0.0
2,2019-05-10,143,5,19,4,NaN,NaN,0.344685,0.378005,0.288359,...,0.000307,0,0,0,0,0,0,0,0,0.0
3,2019-11-11,143,11,46,0,NaN,NaN,0.344685,0.378005,0.288359,...,0.000000,0,0,0,0,0,0,0,0,0.0
4,2020-05-29,143,5,22,4,NaN,NaN,0.344685,0.378005,0.288359,...,0.000002,0,0,0,0,0,0,0,0,0.0


In [18]:
(other_features_df
 .merge(features_df, on=['dt', 'grid_index'])
 .to_csv('features/lag_pipeline_features.csv', index=False))

# Expand train

In [7]:
from competition.metro import make_train

In [8]:
step = 0.2
lat_min, lat_max, lon_min, lon_max = (41.0, 81.5, 19.0, 169.0)
array_of_lats = np.arange(lat_min, lat_max, step).round(1)
array_of_lons = np.arange(lon_min, lon_max, step).round(1)

In [10]:
expanded_train_df = make_train('input/train_raw.csv', array_of_lons, array_of_lats, step, '2018-01-01')
expanded_train_df.info()

load_and_prepare_targets
make_and_prepare_grid_df
make_df_only_land
make_df_not_land
Убираем ячейки не из России
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053086 entries, 0 to 1053085
Data columns (total 19 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   dt            1053086 non-null  datetime64[ns]
 1   lon_min       1053086 non-null  float64       
 2   lat_min       1053086 non-null  float64       
 3   lon_max       1053086 non-null  float64       
 4   lat_max       1053086 non-null  float64       
 5   lon           118764 non-null   float64       
 6   lat           118764 non-null   float64       
 7   grid_index    1053086 non-null  int64         
 8   type_id       118764 non-null   float64       
 9   type_name     118764 non-null   object        
 10  is_land       1053086 non-null  bool          
 11  infire_day_1  1053086 non-null  int64         
 12  infire_day_2  1053086 non-null  int64    

In [11]:
expanded_train_df.to_csv('input/expanded_train.csv', index=False)

# Collect features

In [7]:
from competition.warehouse import Warehouse
from competition.featurise import Featuriser

In [8]:
engine = (Warehouse(data_path=config['warehouse']['data_path'],
                    train=True)
          .create(config['warehouse']['create_args']))
engine

In [10]:
features_df = Featuriser(engine, repo).get_features(config['featurise']['calcers'])
features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488103 entries, 0 to 488102
Columns: 219 entries, dt to infire_day_num
dtypes: float32(200), float64(4), int64(12), object(3)
memory usage: 446.9+ MB


In [11]:
features_df.columns

Index(['dt', 'grid_index', 'month', 'week', 'day_of_week', 'geo_population',
       'geo_place', 'geo_cn5_city_mean_distance', 'geo_cn5_city_max_distance',
       'geo_cn5_city_min_distance',
       ...
       'wind_tp_ws3_mean_lag_3', 'infire_day_1', 'infire_day_2',
       'infire_day_3', 'infire_day_4', 'infire_day_5', 'infire_day_6',
       'infire_day_7', 'infire_day_8', 'infire_day_num'],
      dtype='object', length=219)

In [12]:
features_df.head()

,dt,grid_index,month,week,day_of_week,geo_population,geo_place,geo_cn5_city_mean_distance,geo_cn5_city_max_distance,geo_cn5_city_min_distance,...,wind_tp_ws3_mean_lag_3,infire_day_1,infire_day_2,infire_day_3,infire_day_4,infire_day_5,infire_day_6,infire_day_7,infire_day_8,infire_day_num
0,2020-05-04,143,5,19,0,NaN,NaN,0.344685,0.378005,0.288359,...,2.064614e-04,0,0,0,0,0,0,0,0,0.0
1,2021-02-24,891,2,8,2,NaN,NaN,0.290611,0.404587,0.164580,...,1.102550e-03,0,0,0,0,0,0,0,0,0.0
2,2021-02-27,891,2,8,5,NaN,NaN,0.290611,0.404587,0.164580,...,3.826576e-03,0,0,0,0,0,0,0,0,0.0
3,2021-04-01,892,4,13,3,2054,village,0.216737,0.292387,0.096216,...,9.598749e-06,0,0,0,0,0,0,0,0,0.0
4,2020-03-14,893,3,11,5,None,village,0.189796,0.267262,0.103796,...,8.523463e-07,0,0,0,0,0,0,0,0,0.0


In [13]:
features_df.describe(include='all').T.tail(20)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
wind_v10_ws3_mean_lag_2,488024,NaN,NaN,NaN,0.155824,2.26971,-15.2033,-1.30447,0.147992,1.57001,19.2576
wind_tp_ws3_max_lag_2,488024,NaN,NaN,NaN,0.000941589,0.00215276,0,1.34669e-06,0.000117246,0.000844103,0.0952358
wind_tp_ws3_mean_lag_2,488024,NaN,NaN,NaN,0.000707508,0.00173172,0,8.80141e-07,6.79918e-05,0.000575052,0.0819341
wind_sp_ws3_max_lag_3,488024,NaN,NaN,NaN,98350.5,3389.2,73235.9,97092.4,99357.7,100609,105527
wind_sp_ws3_mean_lag_3,488024,NaN,NaN,NaN,97586.9,3950.81,72082.9,96087.4,98856.8,100252,105359
wind_u10_ws3_max_lag_3,488024,NaN,NaN,NaN,0.732242,2.36844,-16.4305,-0.752289,0.801353,2.05615,19.3251
wind_u10_ws3_mean_lag_3,488024,NaN,NaN,NaN,0.371752,2.35305,-17.1395,-1.10282,0.400017,1.76687,17.9639
wind_v10_ws3_max_lag_3,488024,NaN,NaN,NaN,0.549664,2.29223,-15.6073,-0.962599,0.611069,1.90231,19.319
wind_v10_ws3_mean_lag_3,488024,NaN,NaN,NaN,0.179181,2.27675,-16.0265,-1.29002,0.169922,1.59368,18.8487
wind_tp_ws3_max_lag_3,488024,NaN,NaN,NaN,0.00096009,0.00217655,0,1.70469e-06,0.000123225,0.000864309,0.111974


In [14]:
features_df.to_csv('features.csv', index=False)

# Pipeline

In [7]:
features_df = pd.read_csv('features.csv')
features_df.head()

,dt,grid_index,month,week,day_of_week,geo_population,geo_place,geo_cn5_city_mean_distance,geo_cn5_city_max_distance,geo_cn5_city_min_distance,infire_day_1,infire_day_2,infire_day_3,infire_day_4,infire_day_5,infire_day_6,infire_day_7,infire_day_8,infire_day_num
0,2020-05-04,143,5,19,0,NaN,NaN,0.344685,0.378005,0.288359,0,0,0,0,0,0,0,0,0.0
1,2021-02-24,891,2,8,2,NaN,NaN,0.290611,0.404587,0.164580,0,0,0,0,0,0,0,0,0.0
2,2021-02-27,891,2,8,5,NaN,NaN,0.290611,0.404587,0.164580,0,0,0,0,0,0,0,0,0.0
3,2021-04-01,892,4,13,3,2054.0,village,0.216737,0.292387,0.096216,0,0,0,0,0,0,0,0,0.0
4,2020-03-14,893,3,11,5,NaN,village,0.189796,0.267262,0.103796,0,0,0,0,0,0,0,0,0.0


In [8]:
pipeline = repo.get_object(config['pipeline']['name'])(config['pipeline']['seed'])
pipeline

In [9]:
pipeline.fit(features_df, config['pipeline'], repo)

[Pipeline] ...... (step 1 of 3) Processing transformers, total=   0.1s
[Pipeline] .... (step 2 of 3) Processing dummy_selector, total=   0.4s
[Pipeline] ............. (step 3 of 3) Processing model, total=   4.0s
[Pipeline] ...... (step 1 of 3) Processing transformers, total=   0.1s
[Pipeline] .... (step 2 of 3) Processing dummy_selector, total=   0.4s
[Pipeline] ............. (step 3 of 3) Processing model, total=   0.9s
[Pipeline] ...... (step 1 of 3) Processing transformers, total=   0.1s
[Pipeline] .... (step 2 of 3) Processing dummy_selector, total=   0.4s
[Pipeline] ............. (step 3 of 3) Processing model, total=   0.9s
[Pipeline] ...... (step 1 of 3) Processing transformers, total=   0.1s
[Pipeline] .... (step 2 of 3) Processing dummy_selector, total=   0.4s
[Pipeline] ............. (step 3 of 3) Processing model, total=   1.0s
[Pipeline] ...... (step 1 of 3) Processing transformers, total=   0.1s
[Pipeline] .... (step 2 of 3) Processing dummy_selector, total=   0.4s
[Pipel

In [10]:
pipeline.save('models/test')

In [11]:
pipeline = (repo
            .get_object(config['pipeline']['name'])
            .load('models/test', config['pipeline']['seed']))
pipeline.models

{'infire_day_num': Pipeline(steps=[('transformers',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('geo_place',
                                                   Pipeline(steps=[('fillna_transformer',
                                                                    SimpleImputer(fill_value='NaN',
                                                                                  strategy='constant'))]),
                                                   ['geo_place'])])),
                 ('dummy_selector', DummySelector()),
                 ('model',
                  <catboost.core.CatBoostClassifier object at 0x7f3d0fe92790>)],
          verbose=True),
 'infire_day_1': Pipeline(steps=[('transformers',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('geo_place',
                                                   Pipeline(steps=[('fillna_transformer',
    

In [12]:
y_pred = pipeline.predict(features_df[features_df.columns[2:-9]])
y_pred

,infire_day_1,infire_day_2,infire_day_3,infire_day_4,infire_day_5,infire_day_6,infire_day_7,infire_day_8
0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1
3,1,1,1,1,1,1,1,1
4,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
488098,0,0,0,0,0,0,0,0
488099,0,0,0,0,0,0,0,0
488100,0,0,0,0,0,0,0,0
488101,0,0,0,0,0,0,0,0


In [14]:
y_true = features_df[features_df.columns[-9:-1]]
y_true

,infire_day_1,infire_day_2,infire_day_3,infire_day_4,infire_day_5,infire_day_6,infire_day_7,infire_day_8
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
488098,0,0,0,0,0,0,0,0
488099,0,0,0,0,0,0,0,0
488100,0,0,0,0,0,0,0,0
488101,0,0,0,0,0,0,0,0


In [15]:
from competition.metrics import competition_metric
competition_metric(y_true, y_pred)

0.89305

In [6]:
import pandas as pd
test_df = pd.read_csv('input/test.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488103 entries, 0 to 488102
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   dt            488103 non-null  object 
 1   lon_min       488103 non-null  float64
 2   lat_min       488103 non-null  float64
 3   lon_max       488103 non-null  float64
 4   lat_max       488103 non-null  float64
 5   lon           56927 non-null   float64
 6   lat           56927 non-null   float64
 7   grid_index    488103 non-null  int64  
 8   type_id       56927 non-null   float64
 9   type_name     56927 non-null   object 
 10  is_land       488103 non-null  bool   
 11  infire_day_1  488103 non-null  int64  
 12  infire_day_2  488103 non-null  int64  
 13  infire_day_3  488103 non-null  int64  
 14  infire_day_4  488103 non-null  int64  
 15  infire_day_5  488103 non-null  int64  
 16  infire_day_6  488103 non-null  int64  
 17  infire_day_7  488103 non-null  int64  
 18  infi

In [7]:
test_df.sample(100).to_csv('input/test.csv', index=False)